In [11]:
# Import packages
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import iaml01cw2_helpers as hp
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, log_loss, mean_squared_error
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from scipy.cluster import hierarchy
KNeighboursClassifier = KNeighborsClassifier # For the Brits!
%matplotlib inline

In [2]:
Xtrn,Ytrn,Xtst,Ytst = hp.load_CoVoST2('D:\iaml-cw2\data')

In [1]:
df=pd.DataFrame(Ytrn,columns=['y'])

In [2]:
#3.1
km=KMeans(22,random_state=1)
km.fit(Xtrn,Ytrn)
km.inertia_

In [4]:
#3.1
df=pd.DataFrame(km.labels_)
for i in range (22):
    print('Number of Samples in Cluster '+str(i)+':'+str(df[df[0]==i].shape[0]))

In [6]:
dfx=pd.DataFrame(Xtrn)
dfy=pd.DataFrame(Ytrn,columns=['y'])
frames=[dfx,dfy]
dft=pd.concat(frames,axis=1)

In [5]:
#3.2
pca=PCA(2)
pca.fit(Xtrn,Ytrn)

In [6]:
#3.2
ms=[[0 for i in range (22)]for j in range (2)]
for i in range (22):
    c=dft[dft['y']==i].drop(columns=['y'])
    m=c.mean().dot(pca.components_.transpose())
    ms[0][i]=m[0]
    ms[1][i]=m[1]
plt.scatter(ms[0],ms[1],s=20)
for i in range (22):
    plt.annotate(str(i),xy=(ms[0][i],ms[1][i]))
cc2d=km.cluster_centers_.dot(pca.components_.transpose()).transpose()
plt.scatter(cc2d[0],cc2d[1],s=20)
for i in range (22):
    plt.annotate(str(i),xy=(cc2d[0][i],cc2d[1][i]))
plt.legend(labels=['Mean Vector of PCA','Cluster Center'])
plt.title('Mean Vectors and Cluster Centers in 2D')
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.savefig('3-2.png')

In [3]:
dfx=pd.DataFrame(Xtrn)
dfy=pd.DataFrame(Ytrn,columns=['y'])
frames=[dfx,dfy]
dft=pd.concat(frames,axis=1)
x0=dft[dft['y']==0].drop(columns=['y'])
y0=dft[dft['y']==0][['y']]
dfxt=pd.DataFrame(Xtst)
dfyt=pd.DataFrame(Ytst,columns=['y'])
frames=[dfxt,dfyt]
dftt=pd.concat(frames,axis=1)
x0t=dftt[dftt['y']==0].drop(columns=['y'])
y0t=dftt[dftt['y']==0][['y']]

In [11]:
#3.5
k=[1,3,5,10,15]
full_test=[0 for i in range(5)]
diagonal_test=[0 for i in range(5)]
full_train=[0 for i in range(5)]
diagonal_train=[0 for i in range(5)]
for i in range (5):
    gm1=GaussianMixture(k[i],covariance_type='full',random_state=1)
    gm1.fit(x0,y0)
    full_train[i]=gm1.score(x0,y0)
    full_test[i]=gm1.score(x0t,y0t)
    gm2=GaussianMixture(k[i],covariance_type='diag',random_state=1)
    gm2.fit(x0,y0)
    diagonal_train[i]=gm2.score(x0,y0)
    diagonal_test[i]=gm2.score(x0t,y0t)

In [7]:
#3.5
plt.plot(k,diagonal_train)
plt.plot(k,full_train)
plt.plot(k,diagonal_test)
plt.plot(k,full_test)
plt.legend(labels=['Diagonal_train','Full_train','Diagonal_test','Full_test'])
plt.title('Relationship Between Log-likelihood and K')
plt.xlabel('Number of mixtures')
plt.xticks([1,3,5,10,15],['K=1','K=3','K=5','K=10','K=15'])
plt.ylabel('Per-sample average Log-likelihood')
plt.savefig('3-5.png')

In [8]:
#3.5
print(diagonal_train)
print(diagonal_test)
print(full_train)
print(full_test)

In [9]:
#3.3
m=[dft[dft['y']==i].drop(columns=['y']).mean() for i in range (22)]
hierarchy.dendrogram(hierarchy.ward(m),orientation='right')
plt.ylabel('Languages')
plt.title('Hierarchy of Languages')
plt.xlabel('Distance Dissimilarity')
plt.savefig('3-3.png')

In [10]:
#3.4
a=[]
for i in range (22):
    c=dft[dft['y']==i].drop(columns=['y'])
    km=KMeans(3,random_state=1)
    km.fit(c)
    for j in range (3):
        a.append(km.cluster_centers_[j])
l=[str(int(i/3))+','+str(i%3) for i in range (66)]
hierarchy.dendrogram(hierarchy.ward(a),orientation='right',labels=l)
plt.ylabel('Languages and Clusters')
plt.title('Hierarchy Using Ward Link ')
plt.xlabel('Distance Dissimilarity')
plt.savefig('3-4-ward.png')

In [11]:
#3.4
hierarchy.dendrogram(hierarchy.single(a),orientation='right',labels=l)
plt.ylabel('Languages and Clusters')
plt.title('Hierarchy Using Single Link ')
plt.xlabel('Distance Dissimilarity')
plt.savefig('3-4-single.png')

In [12]:
#3.4
hierarchy.dendrogram(hierarchy.complete(a),orientation='right',labels=l)
plt.ylabel('Languages and Clusters')
plt.title('Hierarchy Using Complete Link ')
plt.xlabel('Distance Dissimilarity')
plt.savefig('3-4-complete.png')